In [1]:
#Columns are Purchase ID, SN, Age, Gender, Item ID, Item Name, Price
#dependencies
import pandas as pd

#Store filepath as variable
filepath = "purchase_data.csv"

#pull into a data frame
df = pd.read_csv(filepath)
df2 = df
#df.head()

In [2]:
#Count total number of players
uniquesn = df["SN"].nunique()
uniqued = {'Total Players': [uniquesn]}
total_players = [uniquesn]
uniquedf = pd.DataFrame(uniqued)
display(uniquedf)

,Total Players
0,576


In [3]:
#Purchasing Analysis (Total)
#number of unique items
unique_items = df["Item ID"].nunique()

#total number of purchases
unique_purchases = df["Purchase ID"].nunique()

#total revenue
total_spent = df["Price"].sum()

#average purchase price
average_purchase_price = total_spent / unique_purchases

#Format into dollars and cents
#Steps found here https://stackoverflow.com/questions/21208376/converting-float-to-dollars-and-cents
total_spent = '${0:,.2f}'.format(total_spent)
average_purchase_price = '${0:,.2f}'.format(average_purchase_price)

#organize the data
pa_total_d = {'Number of Unique Items': [unique_items], 
              'Average Price': [average_purchase_price],
              'Number of Purchases': [unique_purchases],
              'Total Revenue': [total_spent]}

#put data into a data frame
pa_total_df = pd.DataFrame(pa_total_d)
display(pa_total_df)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [4]:
#Gender demographics
#Percentage and Count of Male players
#Percentage and count of female players
#Percentage and count of other/non-disclosed

#grab the variable for player count created in the last cell
#display(total_players)

#https://stackoverflow.com/questions/21402485/pandas-how-to-filter-a-df-to-get-unique-entries
#create a data frame with information grouped by gender
gender_unique = df.groupby(["SN"]).first()

#find the number of male players; calculate the percentage
male_players = gender_unique.loc[gender_unique["Gender"] == "Male", :]
male_players = male_players["Gender"].count()
male_percentage = ((male_players / total_players)*100).sum()

#repeat the process for female players
female_players = gender_unique.loc[gender_unique["Gender"] == "Female", :]
female_players = female_players["Gender"].count()
female_percentage = ((female_players / total_players)*100).sum()

#repeat the process for other players
other_players = gender_unique.loc[gender_unique["Gender"] == "Other / Non-Disclosed", :]
other_players = other_players["Gender"].count()
other_percentage = ((other_players / total_players)*100).sum()

#https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.from_dict.html
#organize the data
gender_output_df = {'Male': [male_players, male_percentage],
                    'Female': [female_players, female_percentage],
                    'Other/Non-disclosed': [other_players, other_percentage]}
#put into a data frame
gender_output_df = pd.DataFrame.from_dict(gender_output_df, orient='index',
                                         columns = ["Total Count", "Percentage of Players"])

#https://stackoverflow.com/questions/23981601/format-certain-floating-dataframe-columns-into-percentage-in-pandas
gender_output_df = gender_output_df.style.format({
    "Percentage of Players": "{:.2f}%".format})
display(gender_output_df)


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-disclosed,11,1.91%


In [5]:
#purchasing analysis (gender)
#Break down all below by gender
#Purchase count
#average purchase price
#total purchase value
#average purchase total per person by gender
pa_gender_df = df[["Gender","Purchase ID"]]
pa_gender_df.columns = ['Gender', 'Purchase Count']
pa_gender_df = pa_gender_df.groupby(["Gender"]).count()

total_purch_value = df[["Gender", "Price"]]
total_purch_value.columns = ['Gender', 'Price']
total_purch_value = total_purch_value.groupby(["Gender"]).sum()

merged_gender_pa = pd.merge(left=pa_gender_df, right=total_purch_value, on = "Gender")
merged_gender_pa["Average Purchase Price"] = merged_gender_pa["Price"]/merged_gender_pa["Purchase Count"]

gender_unique_two = df.groupby(["SN"]).first()
gender_unique_two = gender_unique_two.groupby("Gender").count()
gender_unique_two = gender_unique_two[["Purchase ID"]]
gender_unique_two.columns = ['Unique Users']
#display(gender_unique_two)

merged_gender_pa = pd.merge(left=merged_gender_pa, right=gender_unique_two, on = "Gender")
merged_gender_pa["Avg Per Person"] = merged_gender_pa["Price"]/merged_gender_pa["Unique Users"]
merged_gender_pa = merged_gender_pa[["Purchase Count","Average Purchase Price",
                                     "Price","Avg Per Person"]]
merged_gender_pa.columns = ["Purchase Count", "Average Purchase Price",
                            "Total Purchase Value", "Avg Total Purchase per Person"]

merged_gender_pa["Average Purchase Price"] = merged_gender_pa["Average Purchase Price"].map("${:.2f}".format)
merged_gender_pa["Total Purchase Value"] = merged_gender_pa["Total Purchase Value"].map("${:.2f}".format)
merged_gender_pa["Avg Total Purchase per Person"] = merged_gender_pa["Avg Total Purchase per Person"].map("${:.2f}".format)
display(merged_gender_pa)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
#age demographics
#break into bins of 4 years (ie <10, 10-14, 15-19, etc.)
age_demo = df
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_unique_id = gender_unique

age_unique_id["Age Range"] = pd.cut(age_unique_id["Age"], bins, labels = group_names)
age_unique_id = age_unique_id[["Age Range", "Age"]]
age_unique_id = age_unique_id.groupby("Age Range").count()
age_unique_id.columns = ["Total Count"]
age_unique_id["Percentage of Players"] = (age_unique_id["Total Count"] / total_players)*100

age_unique_id["Percentage of Players"] = age_unique_id["Percentage of Players"].map("{:.2f}%".format)
display(age_unique_id)

#Purchase count
#Average Purchase Price
#Total Purchase value
#Average purchase total per person by age group

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [7]:
#add the bins to the age_demo df
age_demo["Age Range"] = pd.cut(age_demo["Age"], bins, labels = group_names)
#display(age_demo)

#Purchase count
age_purchase_count = age_demo[["Purchase ID","Age Range"]]
age_purchase_count = age_purchase_count.groupby("Age Range").count()
age_purchase_count = age_purchase_count.rename(columns = {"Purchase ID": "Purchase Count"})
#display(age_purchase_count)

#Total Purchase value
age_purchase_price = age_demo[["Age Range","Price"]]
age_purchase_price = age_purchase_price.groupby("Age Range").sum()
age_purchase_price = age_purchase_price.rename(columns = {"Price": "Total Purchase Value"})
#display(age_purchase_price)

#Average Purchase Price
age_combined = pd.merge(age_purchase_count, age_purchase_price, on = "Age Range", how = "outer")
age_combined["Average Purchase Price"] = age_combined["Total Purchase Value"]/age_combined["Purchase Count"]
#display(age_combined)

#Average purchase total per person by age group
age_final = pd.merge(age_combined, age_unique_id, on = "Age Range", how = "outer")
age_final["Avg Total Purchase per Person"] = age_final["Total Purchase Value"]/age_final["Total Count"]
age_final = age_final[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]

age_final["Average Purchase Price"] = age_final["Average Purchase Price"].map("${:.2f}".format)
age_final["Total Purchase Value"] = age_final["Total Purchase Value"].map("${:.2f}".format)
age_final["Avg Total Purchase per Person"] = age_final["Avg Total Purchase per Person"].map("${:.2f}".format)

display(age_final)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [8]:
#Top spenders
#identify the 5 top spenders in the game by total purchase value, then list in
#a table:
top_spenders = df
top_spenders = top_spenders[["SN", "Purchase ID", "Price"]]
top_spenders = top_spenders.rename(columns = {"Purchase ID": "Purchase Count",
                                              "Price": "Total Purchase Value"})
top_spenders_price = top_spenders[["SN", "Total Purchase Value"]].groupby("SN").sum()
top_spenders_price = pd.DataFrame(top_spenders_price)
top_spenders_price = top_spenders_price.sort_values("Total Purchase Value", ascending = False)
top_spenders_count = pd.DataFrame(top_spenders[["SN","Purchase Count"]].groupby("SN").count())
top_spenders_final = pd.merge(top_spenders_price, top_spenders_count, on = "SN", how = "outer")
top_spenders_final["Average Purchase Price"] = top_spenders_final["Total Purchase Value"]/top_spenders_final["Purchase Count"]
top_spenders_final = top_spenders_final[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

top_spenders_final["Average Purchase Price"] = top_spenders_final["Average Purchase Price"].map("${:.2f}".format)
top_spenders_final["Total Purchase Value"] = top_spenders_final["Total Purchase Value"].map("${:.2f}".format)
display(top_spenders_final[:5])
#Sn
#Purchase Count
#Average Purchase Price
#total Purchase value
#How to show only first 5 found here:
#https://riptutorial.com/pandas/example/21739/get-the-first-last-n-rows-of-a-dataframe
#I realize I could have just used the head function, but I was curious how to do it for a number
#other than five

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [9]:
#Most popular items
#identify the 5 most popular items by purchase count, then list in a table:
#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value
top_items = pd.DataFrame(df[["Item ID", "Item Name", "Purchase ID", "Price"]])
top_items = top_items.rename(columns = {"Purchase ID": "Purchase Count",
                                        "Price": "Item Price"})
top_items_count = top_items[["Item ID", "Purchase Count"]].groupby("Item ID").count()
top_items_price = top_items[["Item ID", "Item Price"]].groupby("Item ID").mean()
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
top_items = top_items[["Item ID", "Item Name"]].drop_duplicates()
top_items = pd.merge(top_items, top_items_count, on = "Item ID", how = "outer")
top_items = pd.merge(top_items, top_items_price, on = "Item ID", how = "outer")
top_items["Total Purchase Value"] = top_items["Purchase Count"] * top_items["Item Price"]
top_items = top_items.sort_values("Item ID", ascending = True)
top_items = top_items.sort_values("Purchase Count", ascending = False)
top_items = top_items.set_index(["Item ID", "Item Name"])
top_items["Item Price"] = top_items["Item Price"].map("${:.2f}".format)
top_items["Total Purchase Value"] = top_items["Total Purchase Value"].map("${:.2f}".format)
top_items[:5]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [10]:
#Most profitable items
#Identify the 5 most profitable items by total purchase value, then list in a tab:
#item id
#item name
#purchase count
#item price
#total purchase value


top_items_2 = pd.DataFrame(df2[["Item ID", "Item Name", "Purchase ID", "Price"]])
top_items_2 = top_items_2.rename(columns = {"Purchase ID": "Purchase Count",
                                        "Price": "Item Price"})
top_items_2.head()
top_items_count_2 = top_items_2[["Item ID", "Purchase Count"]].groupby("Item ID").count()
top_items_price_2 = top_items_2[["Item ID", "Item Price"]].groupby("Item ID").mean()
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
top_items_2 = top_items_2[["Item ID", "Item Name"]].drop_duplicates()
top_items_2 = pd.merge(top_items_2, top_items_count_2, on = "Item ID", how = "outer")
top_items_2 = pd.merge(top_items_2, top_items_price_2, on = "Item ID", how = "outer")
top_items_2["Total Purchase Value"] = top_items_2["Purchase Count"] * top_items_2["Item Price"]
top_items_2 = top_items_2.sort_values("Item ID", ascending = True)
top_items_2 = top_items_2.sort_values("Total Purchase Value", ascending = False)
top_items_2 = top_items_2.set_index(["Item ID", "Item Name"])
top_items_2["Item Price"] = top_items_2["Item Price"].map("${:.2f}".format)
top_items_2["Total Purchase Value"] = top_items_2["Total Purchase Value"].map("${:.2f}".format)
top_items_2[:5]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [11]:
#include a written condition of three observable trends based on the data

#found in the readme in GitHub